In [6]:
import warnings
warnings.filterwarnings('ignore') # We can suppress the warnings
from sklearn.utils import shuffle
#https://www.convertcsv.com/geojson-to-csv.htm?utm_content=cmp-true
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import math

import matplotlib.pyplot as plt
from dataprep.eda import plot
from shapely import Point , wkt
from datetime import datetime
from pathlib import Path 
from numpy import asarray
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

In [21]:
#error:'utf-8' codec can't decode byte 0x80 in position 60: invalid start byte that whyI used 
#encoding = 'unicode_escape'
df= pd.read_csv("ppr-2017-dublin.csv",encoding = 'unicode_escape')
 


In [18]:
df.head(10)

,Date of Sale (dd/mm/yyyy),Address,Postal Code,County,Price (),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description
0,01/01/2017,"1 PINTAIL HOUSE, REDCOURT OAKS, SEAFIELD RD EAST",Dublin 3,Dublin,"242,424.00",Yes,No,Second-Hand Dwelling house /Apartment,NaN
1,01/01/2017,"11 PINTAIL HOUSE, REDCOURT OAKS, SEAFIELD RD EAST",Dublin 3,Dublin,"242,424.00",Yes,No,Second-Hand Dwelling house /Apartment,NaN
2,01/01/2017,"124 SEAFIELD RD, CLONTARF, DUBLIN 3",Dublin 3,Dublin,"535,500.00",Yes,No,Second-Hand Dwelling house /Apartment,NaN
3,01/01/2017,"126 SEAFIELD RD, CLONTARF, DUBLIN 3",Dublin 3,Dublin,"630,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN
4,01/01/2017,"128 SEAFIELD RD, CLONTARF, DUBLIN 3",Dublin 3,Dublin,"535,500.00",Yes,No,Second-Hand Dwelling house /Apartment,NaN
5,01/01/2017,"13 PINTAIL HOUSE, REDCOURT OAKS, SEAFIELD RD E...",Dublin 3,Dublin,"271,800.00",Yes,No,Second-Hand Dwelling house /Apartment,NaN
6,01/01/2017,"18 PINTAIL HOUSE, REDCOURT OAKS, SEAFIELD RD EAST",Dublin 3,Dublin,"242,424.00",Yes,No,Second-Hand Dwelling house /Apartment,NaN
7,01/01/2017,"20 PINTAIL HOUSE, REDCOURT OAKS, SEAFIELD RD E...",Dublin 3,Dublin,"360,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN
8,01/01/2017,"25 PINTAIL HOUSE, REDCOURT OAKS, SEAFIELD RD EAST",Dublin 3,Dublin,"148,148.00",Yes,No,Second-Hand Dwelling house /Apartment,NaN
9,01/01/2017,"32 PINTAIL HOUSE, REDCOURT OAKS, SEAFIELD RD E...",Dublin 3,Dublin,"352,800.00",Yes,No,Second-Hand Dwelling house /Apartment,NaN


In [22]:
df.tail()

,Date of Sale (dd/mm/yyyy),Address,Postal Code,County,Price (),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description
17947,29/12/2017,"13 Boyd Avenue, Honeypark, Dun Laoghaire",NaN,Dublin,"640,968.00",No,Yes,New Dwelling house /Apartment,greater than or equal to 38 sq metres and less...
17948,29/12/2017,"14 ST ANTHONY'S ROAD, RIALTO, DUBLIN 8",NaN,Dublin,"215,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN
17949,29/12/2017,"8 ST ANTHONYS RD, RIALTO, DUBLIN 8",Dublin 8,Dublin,"215,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN
17950,30/12/2017,"39 GLENCARRIG DR, FIRHOUSE, DUBLIN 24",Dublin 24,Dublin,"350,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN
17951,30/12/2017,"98 KILDONAN AVE, FINGLAS WEST, DUBLIN 11",Dublin 11,Dublin,"220,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17952 entries, 0 to 17951
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Date of Sale (dd/mm/yyyy)  17952 non-null  object
 1   Address                    17952 non-null  object
 2   Postal Code                11543 non-null  object
 3   County                     17952 non-null  object
 4   Price ()                  17952 non-null  object
 5   Not Full Market Price      17952 non-null  object
 6   VAT Exclusive              17952 non-null  object
 7   Description of Property    17952 non-null  object
 8   Property Size Description  4066 non-null   object
dtypes: object(9)
memory usage: 1.2+ MB


In [24]:
df.describe()

,Date of Sale (dd/mm/yyyy),Address,Postal Code,County,Price (),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description
count,17952,17952,11543,17952,17952,17952,17952,17952,4066
unique,303,17796,23,1,3405,2,2,3,3
top,14/12/2017,"58 FERRY CARRIG RD, COOLOCK, DUBLIN 17",Dublin 15,Dublin,"250,000.00",No,No,Second-Hand Dwelling house /Apartment,greater than or equal to 38 sq metres and less...
freq,256,3,1538,17952,198,17119,13937,13885,3064


In [25]:
df.shape

(17952, 9)

In [26]:
from dataprep.eda import create_report
#df2 = gpd.read_file("test.geojson")
report = create_report(df)# dataprep.eda function generate a report

report.save('Report') # save report to local disk
report.show_browser() # show report in the browser

  0%|                                                   | 0/892 [00:00<?, ?it/s]

Report has been saved to Report.html!
